In [1]:
# RUN THIS
import pandas as pd
import numpy as np

NEO_VER_CSV = 'data/out/neo_ver.csv'
NEO_OUT_JSON = 'data/input/records.json'
CROSS_CSV = 'ne4out.csv'

In [87]:
# RUN THIS (takes 30 sec)
# neov_df = pd.read_csv('../neo4j-import/out/neo_ver.csv')
nu_df = pd.read_csv('../../samples/neo4j-cross/matching_url.csv')
old_df = pd.read_csv('out/non_unique_repo.csv')
len(nu_df) # 2097 unique pkgs


1892

In [88]:
# RUN THIS
# Get unique package managers and how many times they appear
list_pms = []
for pms in nu_df["pkgmans"]:
    for pm in pms[1:-1].split(','):
        list_pms.append(pm)
print(len(list_pms))
q = pd.DataFrame(list_pms)
q.value_counts()

KeyError: 'pkgmans'

In [6]:
# RUN THIS (get data for \label{tbl:cross-pkgs})
# Get packages managers that appear with given package manager
def get_pkgmans(pkgman):
    pkgs = nu_df[nu_df['pkgmans'].str.contains(pkgman)]
    pmns = []
    for pms in pkgs['pkgmans']:
        for pm in pms[1:-1].split(','):
            pmns.append(pm)
    pmns = list(set(pmns))
    pmns.remove(pkgman)
    pmns.sort()
    return pmns

print(f'FreeBSD:    {get_pkgmans("freebsd12")}')
print(f'NetBSD:     {get_pkgmans("netbsd9")}')
print(f'VCPKG:      {get_pkgmans("vcpkg")}')
print(f'Conan:      {get_pkgmans("conan")}')
print(f'OpenBSD:    {get_pkgmans("openbsd69")}')
print(f'Rubygems:   {get_pkgmans("rubygems")}')
print(f'NPM:        {get_pkgmans("npm")}')
print(f'Cargo:      {get_pkgmans("cargo")}')
print(f'Homebrew:   {get_pkgmans("homebrew")}')
print(f'Chromebrew: {get_pkgmans("chromebrew")}')
print(f'Clojars:    {get_pkgmans("clojars")}')
print(f'Alire:      {get_pkgmans("alire")}')
print(f'LuaRocks:   {get_pkgmans("luarocks")}')
print(f'FPM:        {get_pkgmans("fpm")}')
print(f'Nimble:     {get_pkgmans("nimble")}')

# ['openbsd', 'netbsd']
# ...


FreeBSD:    ['cargo', 'clojars', 'conan', 'fpm', 'luarocks', 'netbsd9', 'nimble', 'npm', 'openbsd69', 'rubygems', 'vcpkg']
NetBSD:     ['cargo', 'conan', 'freebsd12', 'luarocks', 'npm', 'openbsd69', 'rubygems', 'vcpkg']
VCPKG:      ['alire', 'cargo', 'conan', 'freebsd12', 'netbsd9', 'npm', 'openbsd69', 'rubygems']
Conan:      ['alire', 'cargo', 'chromebrew', 'freebsd12', 'homebrew', 'netbsd9', 'npm', 'openbsd69', 'rubygems', 'vcpkg']
OpenBSD:    ['cargo', 'clojars', 'conan', 'freebsd12', 'netbsd9', 'npm', 'vcpkg']
Rubygems:   ['cargo', 'clojars', 'conan', 'freebsd12', 'netbsd9', 'npm', 'vcpkg']
NPM:        ['cargo', 'clojars', 'conan', 'freebsd12', 'homebrew', 'luarocks', 'netbsd9', 'openbsd69', 'rubygems', 'vcpkg']
Cargo:      ['clojars', 'conan', 'freebsd12', 'netbsd9', 'npm', 'openbsd69', 'rubygems', 'vcpkg']
Homebrew:   ['chromebrew', 'conan', 'npm']
Chromebrew: ['conan', 'homebrew']
Clojars:    ['cargo', 'freebsd12', 'npm', 'openbsd69', 'rubygems']
Alire:      ['conan', 'vcpkg']
L

In [56]:
# THIS IS OLD MAIN SCRIPT
data_path = '../../dasea_data'
path_dict = {
    'alire': '/alire/alire_dependencies_05-17-2022.csv',
    'cargo': '/cargo/cargo_dependencies_05-17-2022.csv',
    'chromebrew': '/chromebrew/chromebrew_dependencies_05-17-2022.csv',
    'clojars': '/clojars/clojars_dependencies_05-19-2022.csv',
    'conan': '/conan/conan_dependencies_05-17-2022.csv',
    'freebsd12': '/ports/freebsd12/freebsd12_dependencies_05-17-2022.csv',
    'fpm': '/fpm/fpm_dependencies_05-17-2022.csv',
    'homebrew': '/homebrew/homebrew_dependencies_05-17-2022.csv',
    'netbsd9': '/ports/netbsd9/netbsd9_dependencies_05-17-2022.csv',
    'nimble': '/nimble/nimble_dependencies_05-17-2022.csv',
    'openbsd69': '/ports/openbsd69/openbsd69_dependencies_05-17-2022.csv',
    'vcpkg': '/vcpkg/vcpkg_dependencies_05-17-2022.csv',
    'luarocks': '/luarocks/luarocks_dependencies_05-17-2022.csv',
    'npm': '/npm/npm_dependencies_05-17-2022.csv',
    'rubygems': '/rubygems/rubygems_dependencies_05-17-2022.csv'
}

df_dict = {}

def _df(path):
    import os
    file_path = os.path.abspath(data_path + path)
    return pd.read_csv(file_path)


# outfil = pd.read_csv('out/non_unique_repo.csv')

all_urls = nu_df['url'].unique()
res_df = pd.DataFrame(columns=('pkgnames','ecosystem', 'count_deps', 'count_total'))
counter = 0

for url in all_urls[:5]:
    counter += 1
    print(counter)
    # pkgs = neov_df[neov_df['url'] == url]['pkg_name'].unique().tolist()
    pkgnames = neov_df[neov_df['url'] == url]['pkg_name'].unique().tolist()
    # num_pkgs = neov_df[neov_df.duplicated(subset=['pkgman','url'], keep=False)]
    acc_list = []
    pmlst = []
    acc = 0
    for pm in list(set(list_pms)):
        if pm != 'npm': # dirty fix - remove when ready for real data
            path = path_dict[pm]
            ddf = pd.DataFrame()
            if path in df_dict:
                ddf = df_dict[path]
            else:
                ddf = _df(data_path + path)
                df_dict[path] = ddf
            for pkgname in pkgnames:
                targets = ddf[ddf['target_name'] == pkgname]
                if len(targets) > 1:
                    acc = acc + len(targets)
                    acc_list.append(len(ddf[ddf['target_name'] == pkgname]))
                    pmlst.append(pm)
        # else:
        #     acc_list.append(-1)
    
    entry = pd.DataFrame.from_dict({
        "pkgnames": [pkgnames],
        "ecosystem":  [list(set(pmlst))],
        "count_deps": [acc_list],
        "count_total": [acc]
    })
    res_df = pd.concat([res_df, entry], ignore_index=True)



In [127]:
# RUN THIS to export csv
res_df.sort_values(by=['count_total'], ascending=False).to_csv('out/res_df2.csv')

In [12]:
df = pd.read_csv("../../samples/neo4j-cross/matching_url_obj.csv")
df = df.drop(columns=['pkgmans'])
df.to_csv("../../samples/neo4j-cross/matching_url_obj_no_pkgmans.csv", index=False)

In [1]:
# NEW SCRIPT
import pandas as pd
from ast import literal_eval
data_path = '../../dasea_data'
path_dict = {
    'alire': '/alire/alire_dependencies_05-17-2022.csv',
    'cargo': '/cargo/cargo_dependencies_05-17-2022.csv',
    'chromebrew': '/chromebrew/chromebrew_dependencies_05-17-2022.csv',
    'clojars': '/clojars/clojars_dependencies_05-19-2022.csv',
    'conan': '/conan/conan_dependencies_05-17-2022.csv',
    'freebsd12': '/ports/freebsd12/freebsd12_dependencies_05-17-2022.csv',
    'fpm': '/fpm/fpm_dependencies_05-17-2022.csv',
    'homebrew': '/homebrew/homebrew_dependencies_05-17-2022.csv',
    'netbsd9': '/ports/netbsd9/netbsd9_dependencies_05-17-2022.csv',
    'nimble': '/nimble/nimble_dependencies_05-17-2022.csv',
    'openbsd69': '/ports/openbsd69/openbsd69_dependencies_05-17-2022.csv',
    'vcpkg': '/vcpkg/vcpkg_dependencies_05-17-2022.csv',
    'luarocks': '/luarocks/luarocks_dependencies_05-17-2022.csv',
    'npm': '/npm/npm_dependencies_05-17-2022.csv',
    'rubygems': '/rubygems/rubygems_dependencies_05-17-2022.csv'
}

df_dict = {}

def _df(path):
    import os
    file_path = os.path.abspath(data_path + path)
    return pd.read_csv(file_path)

def string_to_list_of_dicts(string):
    return literal_eval(string.replace('":', '":"').replace(',"','","').replace("}", '"}'))

def is_valid(dictionary):
    zero_count = 0
    for key in dictionary:
        if dictionary[key] == 0:
            zero_count += 1
    return len(dictionary) - zero_count > 1

def sum_all_values_in_dict(dictionary):
    return sum(dictionary.values())

res_df = pd.DataFrame(columns=('url', 'count_deps', 'count_total'))
counter = 0

df = pd.read_csv("../../samples/neo4j-cross/multiple-ecosystems-results.csv")

pck_count = 0
for row in df.itertuples():
    counter += 1
    print(counter)
    # if counter > 20: ## REMOVE WHEN READY FOR REAL DATA
    #     break

    url = row[2]
    if "git" not in url:
        continue
    names_and_pkgmans = string_to_list_of_dicts(row[1])
    pkgmans = []
    res = {}
    for name_and_pkgman in names_and_pkgmans:
        pck_count += 1
        name = name_and_pkgman['name']
        pkgman = name_and_pkgman['pkgman']
        # if name != "openssl":
        #     break
        pkgmans.append(pkgman)
        if pkgman not in res:
            res[pkgman] = 0
        
        path = path_dict[pkgman]
        ddf = pd.DataFrame()
        if path in df_dict:
            ddf = df_dict[path]
        else:
            ddf = _df(data_path + path)
            df_dict[path] = ddf
        targets = ddf[ddf['target_name'] == name]
        uniq = targets['source_name'].unique()
        # print(uniq)
        if len(uniq) > 0:
            res[pkgman] = res[pkgman] + len(uniq)
    if is_valid(res):
        entry = pd.DataFrame.from_dict({
            "url": [url],
            "count_deps": [res],
            "count_total": [sum(res.values())]
        })
        res_df = pd.concat([res_df, entry], ignore_index=True)

print(pck_count)



NameError: name 'pd' is not defined

In [91]:
# RUN THIS to export csv
res_df.sort_values(by=['count_total'], ascending=False).to_csv('out/res_df5.csv')